In [1]:
# import necessary libraries
from resemblyzer import preprocess_wav, VoiceEncoder
from  pathlib import Path 
from tqdm import tqdm 
import numpy as np 
from IPython.display import Audio 
from itertools import groupby
import heapq
import torchaudio as ta
from os import listdir
from os.path import isfile, join, basename
from  pathlib import Path 
import numpy as np
import torch

In [2]:
dir_path = "Tracks/vocals_chunks"
audio_files = [join(dir_path, f) for f in listdir(dir_path) if "mp3" in f and isfile(join(dir_path, f))]

resemblyzer_vectors = []
spectrograms = []
failed_files = []
transform = ta.transforms.Spectrogram(158,158,1725)
ta.transforms.Spectrogram()
#use 158, 1725 to get 256 spec
#audio_files = audio_files[0:2]

import json
audio_files = json.load('files.json')

for f in audio_files:
    try:
        vocals = ta.load(f)
        #print(vocals[0].shape)
        mono = torch.mean(vocals[0],0,False)
        #print(mono.shape)

        #print(f)
        wav = preprocess_wav(f)
        print(wav)

        #Get style encoding
        encoder = VoiceEncoder()
        embed = encoder.embed_utterance(wav)
        np.set_printoptions(precision=3, suppress=True)

        #Get Spectrogram
        spec = transform(mono)

        #Add spec and style embedding to appropriate arrays
        resemblyzer_vectors.append(embed)
        spectrograms.append(spec)
    except:
        print('failed :(')
        failed_files.append(f)

resemblyzer_vectors = np.array(resemblyzer_vectors)
spectrograms = np.array(spectrograms)
spectrograms = np.transpose(spectrograms,(0,2,1)) #reshape vectors to align with resemblyzer
print(resemblyzer_vectors.shape)
print(spectrograms.shape)
print("Failed Files: ")
print(failed_files)


FileNotFoundError: [Errno 2] No such file or directory: 'Tracks/vocals_chunks'

In [36]:
print(resemblyzer_vectors)

[[0.    0.    0.003 ... 0.026 0.    0.   ]
 [0.    0.    0.001 ... 0.074 0.002 0.   ]
 [0.    0.    0.    ... 0.14  0.001 0.   ]
 ...
 [0.015 0.001 0.055 ... 0.003 0.1   0.059]
 [0.013 0.004 0.012 ... 0.006 0.077 0.079]
 [0.    0.    0.047 ... 0.    0.108 0.063]]


In [37]:
file_path = "resemblyzer_vectors_hf.npy"
fpath2 = "spectrograms_hf.npy"
np.save(file_path, resemblyzer_vectors)
np.save(fpath2,spectrograms)

In [38]:
resemblyzer_vectors = np.load(file_path)
specs = np.load(fpath2)
print(resemblyzer_vectors.shape)
print(specs.shape)



(2201, 256)
(2201, 256, 80)
